# Modelo de Classificação: Predição de Recompra em 30 dias

Este notebook implementa um modelo de machine learning para prever a probabilidade de recompra de clientes em um horizonte de 30 dias, utilizando dados históricos de transações de uma empresa de transporte rodoviário.

In [27]:
# Importações necessárias
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas de machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Utilitários
import pickle
import os
import json

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [28]:
# Carregamento e análise exploratória dos dados
df = pd.read_csv('files/df_t.csv')

print("Dados carregados com sucesso!")
print(f"Shape do dataset: {df.shape}")
print("\n" + "="*50)
print("PRIMEIRAS 5 LINHAS:")
print(df.head())

print("\n" + "="*50)
print("INFORMAÇÕES GERAIS:")
print(df.info())

print("\n" + "="*50)
print("VALORES NULOS:")
print(df.isnull().sum())

print("\n" + "="*50)
print("ESTATÍSTICAS DESCRITIVAS:")
print(df.describe())

print("\n" + "="*50)
print("AMOSTRA DE DATAS:")
print(df['date_purchase'].head(10))

Dados carregados com sucesso!
Shape do dataset: (1741344, 12)

PRIMEIRAS 5 LINHAS:
                                 nk_ota_localizer_id  \
0  bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...   
1  5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...   
2  fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...   
3  4dc44a6dd592b702feccb493d192210c86965aee684529...   
4  aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...   

                                          fk_contact date_purchase  \
0  a7218ff4ee7d37d48d2b4391b955627cb089870b934912...    2018-12-26   
1  37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...    2018-12-05   
2  3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...    2018-12-21   
3  ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...    2018-12-06   
4  ceea0de820a6379f2c4215bddaec66c33994b304607e56...    2021-02-23   

  time_purchase                             place_origin_departure  \
0      15:33:35  6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...   
1      15:07:57  10

In [29]:
# Análise temporal dos dados
df['date_purchase'] = pd.to_datetime(df['date_purchase'])

print("ANÁLISE TEMPORAL:")
print(f"Data mais antiga: {df['date_purchase'].min()}")
print(f"Data mais recente: {df['date_purchase'].max()}")
print(f"Período total: {(df['date_purchase'].max() - df['date_purchase'].min()).days} dias")

print("\n" + "="*50)
print("ANÁLISE DE CLIENTES:")
total_clientes = df['fk_contact'].nunique()
print(f"Total de clientes únicos: {total_clientes:,}")
print(f"Média de compras por cliente: {len(df) / total_clientes:.2f}")

print("\n" + "="*50)
print("DISTRIBUIÇÃO POR ANO:")
df['year'] = df['date_purchase'].dt.year
print(df['year'].value_counts().sort_index())

print("\n" + "="*50)
print("ANÁLISE DE VALORES:")
print(f"Compras com GMV positivo: {(df['gmv_success'] > 0).sum():,}")
print(f"Compras com GMV negativo: {(df['gmv_success'] < 0).sum():,}")
print(f"Compras com GMV zero: {(df['gmv_success'] == 0).sum():,}")

# Definir data de corte
data_corte = df['date_purchase'].max() - timedelta(days=30)
print(f"\nData de corte para criar target: {data_corte}")
print(f"Registros após data de corte: {(df['date_purchase'] > data_corte).sum():,}")

ANÁLISE TEMPORAL:
Data mais antiga: 2013-09-12 00:00:00
Data mais recente: 2024-04-01 00:00:00
Período total: 3854 dias

ANÁLISE DE CLIENTES:
Total de clientes únicos: 581,817
Média de compras por cliente: 2.99

DISTRIBUIÇÃO POR ANO:
year
2013       341
2014     15182
2015     29654
2016     54622
2017     81354
2018    100277
2019    180935
2020    145669
2021    262649
2022    380073
2023    383163
2024    107425
Name: count, dtype: int64

ANÁLISE DE VALORES:
Compras com GMV positivo: 1,741,159
Compras com GMV negativo: 7
Compras com GMV zero: 178

Data de corte para criar target: 2024-03-02 00:00:00
Registros após data de corte: 34,185


In [30]:
# Preparação do dataset para modelagem preditiva

# Definir data de corte para separar dados históricos dos dados de validação
data_corte = pd.to_datetime('2024-03-01')
data_fim_observacao = data_corte + timedelta(days=30)

print(f"Data de corte: {data_corte.strftime('%Y-%m-%d')}")
print(f"Janela de observação: até {data_fim_observacao.strftime('%Y-%m-%d')}")

print("Criando dataset para modelagem...")

# 1. Separar dados históricos (antes do corte)
dados_historicos = df[df['date_purchase'] < data_corte].copy()
print(f"Compras históricas: {len(dados_historicos):,}")

# 2. Última compra de cada cliente no período histórico
ultima_compra = dados_historicos.groupby('fk_contact').agg({
    'date_purchase': 'max',
    'gmv_success': 'last',
    'total_tickets_quantity_success': 'last',
    'place_origin_departure': 'last',
    'place_destination_departure': 'last',
    'fk_departure_ota_bus_company': 'last'
}).reset_index()

ultima_compra.columns = ['fk_contact', 'data_ultima_compra', 'gmv_ultima_compra', 
                        'tickets_ultima_compra', 'origem_ultima', 'destino_ultima', 'empresa_ultima']

print(f"Clientes únicos: {len(ultima_compra):,}")

# 3. Identificar quem comprou na janela futura para criar target
compras_futuras = df[
    (df['date_purchase'] >= data_corte) & 
    (df['date_purchase'] <= data_fim_observacao)
]['fk_contact'].unique()

print(f"Clientes que compraram na janela futura: {len(compras_futuras):,}")

# 4. Criar variável target
ultima_compra['target'] = ultima_compra['fk_contact'].isin(compras_futuras).astype(int)

# Verificar distribuição do target
target_dist = ultima_compra['target'].value_counts()
print(f"\nDistribuição do Target:")
print(f"Não compraram (0): {target_dist[0]:,} ({target_dist[0]/len(ultima_compra)*100:.1f}%)")
print(f"Compraram (1): {target_dist[1]:,} ({target_dist[1]/len(ultima_compra)*100:.1f}%)")

print(f"\nDataset base: {ultima_compra.shape}")
print(ultima_compra.head())

dataset_base = ultima_compra.copy()
print("\nDataset base criado com sucesso!")

Data de corte: 2024-03-01
Janela de observação: até 2024-03-31
Criando dataset para modelagem...
Compras históricas: 1,705,101
Clientes únicos: 572,993
Clientes que compraram na janela futura: 23,768

Distribuição do Target:
Não compraram (0): 557,768 (97.3%)
Compraram (1): 15,225 (2.7%)

Dataset base: (572993, 8)
                                          fk_contact data_ultima_compra  \
0  0000029b76ad3cf9d86ad430754fb1d4478069affda61e...         2021-01-09   
1  000010ae2e13049769982d9f07de792d92452ff1d124e3...         2022-05-05   
2  00001f68902d3e8d332baa62a69065ce71e7b5a8c850a5...         2018-11-01   
3  00007a5d618cd250d7f05766cfe01a8663a3767f1cd669...         2022-12-04   
4  00008c39885815e42a0bb750cee199cd4da741a5645705...         2021-10-01   

   gmv_ultima_compra  tickets_ultima_compra  \
0              91.02                      1   
1              79.52                      1   
2             169.90                      1   
3             131.49                      1  

In [ ]:
# Engenharia de Features para Modelagem Preditiva

print("Iniciando criação de features comportamentais...")
print(f"Utilizando {len(dados_historicos):,} compras históricas")

# 1. Features de Recência
dataset_base['dias_desde_ultima_compra'] = (data_corte - dataset_base['data_ultima_compra']).dt.days

# 2. Features de Frequência
freq_features = dados_historicos.groupby('fk_contact').agg({
    'date_purchase': ['count', 'nunique'],
    'gmv_success': ['sum', 'mean', 'std', 'min', 'max'],
    'total_tickets_quantity_success': ['sum', 'mean', 'max']
}).round(2)

freq_features.columns = [
    'total_compras', 'dias_unicos_compra',
    'gmv_total', 'gmv_medio', 'gmv_std', 'gmv_min', 'gmv_max',
    'tickets_total', 'tickets_medio', 'tickets_max'
]
freq_features = freq_features.fillna(0)

# 3. Features Temporais e Sazonalidade
dados_historicos['mes'] = dados_historicos['date_purchase'].dt.month
dados_historicos['dia_semana'] = dados_historicos['date_purchase'].dt.dayofweek
dados_historicos['hora'] = dados_historicos['time_purchase'].str[:2].astype(int)

sazon_features = dados_historicos.groupby('fk_contact').agg({
    'mes': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
    'dia_semana': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
    'hora': ['mean', 'std']
}).round(2)

sazon_features.columns = ['mes_preferido', 'dia_semana_preferido', 'hora_media', 'hora_std']
sazon_features = sazon_features.fillna(0)

# 4. Features de Comportamento de Viagem
comportamento_features = dados_historicos.groupby('fk_contact').agg({
    'place_origin_departure': 'nunique',
    'place_destination_departure': 'nunique',
    'fk_departure_ota_bus_company': 'nunique'
})

comportamento_features.columns = ['origens_unicas', 'destinos_unicos', 'empresas_unicas']

# 5. Features de Periodicidade
intervalos = dados_historicos.groupby('fk_contact')['date_purchase'].apply(
    lambda x: x.sort_values().diff().dt.days.mean() if len(x) > 1 else 0
).round(2)

periodicidade_features = pd.DataFrame({
    'intervalo_medio_dias': intervalos,
    'regularidade': dados_historicos.groupby('fk_contact')['date_purchase'].apply(
        lambda x: x.sort_values().diff().dt.days.std() if len(x) > 1 else 0
    ).round(2)
}).fillna(0)

# 6. Consolidação das Features
dataset_final = dataset_base.copy()

for features_df in [freq_features, sazon_features, comportamento_features, periodicidade_features]:
    dataset_final = dataset_final.merge(features_df, left_on='fk_contact', right_index=True, how='left')

dataset_final = dataset_final.fillna(0)

print(f"\nDataset final criado!")
print(f"Shape: {dataset_final.shape}")
print(f"Total de features: {dataset_final.shape[1] - 8}")

feature_cols = [col for col in dataset_final.columns if col not in ['fk_contact', 'data_ultima_compra', 'target']]
print(f"\nEstatísticas das principais features:")
print(dataset_final[feature_cols[:10]].describe())

Iniciando criação de features comportamentais...
Utilizando 1,705,101 compras históricas


In [ ]:
# Preparação dos dados para modelagem

print("Preparando dados para treinamento...")

# Separar features e target
feature_cols = [col for col in dataset_final.columns if col not in ['fk_contact', 'data_ultima_compra', 'target']]
X = dataset_final[feature_cols].copy()
y = dataset_final['target'].copy()

print(f"Features selecionadas: {len(feature_cols)}")
print("Features:", feature_cols)

print(f"\nTipos de dados:")
print(X.dtypes.value_counts())

# Separar colunas numéricas e categóricas
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nColunas numéricas: {len(numeric_cols)}")
print(f"Colunas categóricas: {len(categorical_cols)}")

# Codificação de variáveis categóricas
if categorical_cols:
    print("Aplicando Label Encoding nas variáveis categóricas...")
    le_dict = {}
    
    for col in categorical_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        le_dict[col] = le
        print(f"  {col}: {len(le.classes_)} categorias únicas")

# Verificação da qualidade dos dados
print(f"\nVerificação de qualidade:")
print(f"Valores nulos: {X.isnull().sum().sum()}")

numeric_data = X[numeric_cols] if numeric_cols else X
if len(numeric_data.columns) > 0:
    inf_count = np.isinf(numeric_data.select_dtypes(include=[np.number])).sum().sum()
    print(f"Valores infinitos: {inf_count}")
    X = X.replace([np.inf, -np.inf], 0)

# Divisão treino/teste
print(f"\nDivisão dos dados (80/20):")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Treino: {X_train.shape[0]:,} amostras")
print(f"Teste: {X_test.shape[0]:,} amostras")
print(f"Distribuição treino - Classe 0: {(y_train==0).sum():,}, Classe 1: {(y_train==1).sum():,}")
print(f"Distribuição teste - Classe 0: {(y_test==0).sum():,}, Classe 1: {(y_test==1).sum():,}")

# Treinamento do modelo Random Forest
print(f"\nConfigurando modelo Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=1000,
    min_samples_leaf=500,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Treinando modelo...")
rf_model.fit(X_train, y_train)

# Predições
print("Gerando predições...")
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("\nTreinamento concluído!")

In [ ]:
# Avaliação do modelo

print("AVALIAÇÃO DO MODELO")

print(f"\nAUC-ROC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

print(f"\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

print(f"\nMatriz de Confusão:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Importância das features
print(f"\n" + "=" * 40)
print("IMPORTÂNCIA DAS FEATURES")
print("=" * 40)
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 features mais importantes:")
print(feature_importance.head(10))

In [ ]:
# Exportação dos resultados

print("Exportando artefatos do modelo...")

# Criar estrutura de pastas
base_dir = 'dist/classification'
artifacts_dir = os.path.join(base_dir, 'artifacts')

os.makedirs(base_dir, exist_ok=True)
os.makedirs(artifacts_dir, exist_ok=True)

print(f"Estrutura de pastas criada: {base_dir}/")

# 1. Salvar modelo e artefatos
print("\n1. Salvando modelo e componentes...")

model_artifacts = {
    'model': rf_model,
    'feature_columns': feature_cols,
    'label_encoders': le_dict if 'le_dict' in locals() else {},
    'data_corte': data_corte,
    'trained_date': datetime.now(),
    'model_version': 'RandomForest_v1.0',
    'performance_metrics': {
        'auc_roc': round(roc_auc_score(y_test, y_pred_proba), 4),
        'accuracy': round((y_pred == y_test).mean(), 4),
        'precision_class_1': round(classification_report(y_test, y_pred, output_dict=True)['1']['precision'], 4),
        'recall_class_1': round(classification_report(y_test, y_pred, output_dict=True)['1']['recall'], 4),
        'test_samples': len(y_test)
    },
    'feature_importance': feature_importance.to_dict('records')
}

# Salvar modelo
model_path = os.path.join(artifacts_dir, 'modelo_recompra_30dias.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model_artifacts, f)

print(f"Modelo salvo em: {model_path}")

# Salvar importância das features
feature_importance_path = os.path.join(artifacts_dir, 'feature_importance.csv')
feature_importance.to_csv(feature_importance_path, index=False)
print(f"Feature importance salva em: {feature_importance_path}")

# 2. Criar dataset com predições
print("\n2. Criando dataset com predições...")

dataset_completo = dataset_final.copy()

# Preparar dados para predição
X_full = dataset_final[feature_cols].copy()
if 'le_dict' in locals():
    for col in categorical_cols:
        if col in le_dict:
            X_full[col] = le_dict[col].transform(X_full[col].astype(str))

# Fazer predições para todo o dataset
dataset_completo['probabilidade_compra'] = rf_model.predict_proba(X_full)[:, 1]
dataset_completo['predicao_compra'] = rf_model.predict(X_full)

# Criar categorias de potencial
dataset_completo['potencial_recompra'] = pd.cut(
    dataset_completo['probabilidade_compra'], 
    bins=[0, 0.1, 0.3, 0.6, 1.0],
    labels=['Baixo', 'Médio', 'Alto', 'Muito Alto']
)

# Adicionar informações temporais
dataset_completo['mes_ultima_compra'] = dataset_completo['data_ultima_compra'].dt.month
dataset_completo['ano_ultima_compra'] = dataset_completo['data_ultima_compra'].dt.year
dataset_completo['data_predicao'] = datetime.now()
dataset_completo['versao_modelo'] = 'RandomForest_v1.0'

# Selecionar colunas para exportação
colunas_dataset = [
    'fk_contact', 'data_ultima_compra', 'target', 
    'probabilidade_compra', 'predicao_compra', 'potencial_recompra',
    'gmv_ultima_compra', 'tickets_ultima_compra',
    'dias_desde_ultima_compra', 'total_compras', 'gmv_total', 'gmv_medio',
    'mes_ultima_compra', 'ano_ultima_compra',
    'origens_unicas', 'destinos_unicos', 'empresas_unicas',
    'intervalo_medio_dias', 'regularidade', 'data_predicao', 'versao_modelo'
]

dataset_final_export = dataset_completo[colunas_dataset].copy()
dataset_final_export['potencial_recompra'] = dataset_final_export['potencial_recompra'].astype(str)
dataset_final_export['probabilidade_compra'] = dataset_final_export['probabilidade_compra'].round(6)

# Salvar CSV
csv_path = os.path.join(base_dir, 'dataset_recompra_completo.csv')
dataset_final_export.to_csv(csv_path, index=False)

print(f"Dataset preparado: {dataset_final_export.shape[0]:,} registros")
print(f"Dataset salvo em: {csv_path}")

# 3. Criar metadados
print("\n3. Criando arquivo de metadados...")

potencial_stats = dataset_final_export['potencial_recompra'].value_counts().to_dict()

metadata = {
    'projeto': 'Modelo de Classificação - Predição de Recompra em 30 dias',
    'versao_modelo': 'RandomForest_v1.0',
    'data_criacao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_corte': data_corte.strftime('%Y-%m-%d'),
    'dataset': {
        'total_clientes': len(dataset_final_export),
        'total_features': len(feature_cols)
    },
    'performance': {
        'auc_roc': round(roc_auc_score(y_test, y_pred_proba), 4),
        'accuracy': round((y_pred == y_test).mean(), 4),
        'precision_class_1': round(classification_report(y_test, y_pred, output_dict=True)['1']['precision'], 4),
        'recall_class_1': round(classification_report(y_test, y_pred, output_dict=True)['1']['recall'], 4)
    },
    'distribuicao_potencial': {
        'Baixo': int(potencial_stats.get('Baixo', 0)),
        'Médio': int(potencial_stats.get('Médio', 0)),
        'Alto': int(potencial_stats.get('Alto', 0)),
        'Muito Alto': int(potencial_stats.get('Muito Alto', 0))
    },
    'features_importantes': [
        {'feature': row['feature'], 'importance': round(row['importance'], 4)}
        for _, row in feature_importance.head(10).iterrows()
    ],
    'arquivos_gerados': {
        'modelo_completo': 'artifacts/modelo_recompra_30dias.pkl',
        'feature_importance': 'artifacts/feature_importance.csv',
        'dataset_csv': 'dataset_recompra_completo.csv',
        'metadados': 'metadata.json'
    }
}

# Salvar metadados
metadata_path = os.path.join(base_dir, 'metadata.json')
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"Metadados salvos em: {metadata_path}")

print(f"\nDistribuição por potencial de recompra:")
for categoria in ['Muito Alto', 'Alto', 'Médio', 'Baixo']:
    count = potencial_stats.get(categoria, 0)
    pct = count/len(dataset_final_export)*100 if len(dataset_final_export) > 0 else 0
    print(f"  {categoria:12}: {count:6,} clientes ({pct:5.1f}%)")

print(f"\nPerformance do Modelo:")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"Recall (classe 1): {classification_report(y_test, y_pred, output_dict=True)['1']['recall']:.2%}")

print(f"\nTop 5 features mais importantes:")
for i, (_, row) in enumerate(feature_importance.head(5).iterrows(), 1):
    print(f"  {i}. {row['feature']}: {row['importance']:.4f}")

print(f"\nExportação concluída com sucesso!")